In [20]:
import geopandas as gpd
import ee
import pandas as pd
from datetime import datetime
from io import StringIO


# Inicializar a API do Google Earth Engine
ee.Authenticate()
ee.Initialize()

# Carregar o arquivo KML usando GeoPandas
kml_file = "/home/talita/Documentos/calculo_climatologia/calculo_python/kml/areaDesmat_Area1.kml"
gdf = gpd.read_file(kml_file, driver='KML')

# Extraindo a geometria da área de interesse (supondo que há uma única geometria no KML)
area_de_interesse = gdf.geometry[0]

# Converter a geometria para o formato GeoJSON que o GEE pode entender
coords = [list(point)[:2] for point in area_de_interesse.exterior.coords]
geojson_geometry = ee.Geometry.Polygon(coords)

def calcula_pentada(dataInicio, dataFim, geometry):
    # Base utilizada para a coleta de dados
    dataset = ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD').filterDate(dataInicio, dataFim)

    # Selecionar a banda de precipitação
    precipitation = dataset.select('precipitation')

    # Criar um dicionário para armazenar os valores de precipitação por ano
    valores_por_ano = {}
    datas = []  # Lista para armazenar as datas

    # Percorrer a lista de imagens e obter as datas e valores de precipitação
    for i in range(precipitation.size().getInfo()):
        image = ee.Image(precipitation.toList(precipitation.size()).get(i))
        data = ee.Date(image.get('system:time_start')).format('yyyy-MM-dd').getInfo()  # Extrai a data
        ano = data.split('-')[0]

        # Calcular o valor de precipitação na área de interesse (definida pelo KML)
        valor = image.reduceRegion(ee.Reducer.sum(), geometry).get('precipitation').getInfo()

        # Armazenar o valor e a data no dicionário
        if ano not in valores_por_ano:
            valores_por_ano[ano] = []
        valores_por_ano[ano].append(valor)
        datas.append(data)  # Adiciona a data à lista

    # Criar um DataFrame a partir do dicionário de valores por ano
    df_pentada = pd.DataFrame.from_dict(valores_por_ano, orient='index').transpose()

    # Adicionar uma coluna de Pentada com a contagem correta
    df_pentada["Pentada"] = [f"{i+1}º" for i in range(df_pentada.shape[0])]

    # Adicionar a coluna de datas das pentadas
    df_pentada["Data"] = datas  # Usar a lista de datas extraídas

    return df_pentada


# Definir datas de início e fim 
dataInicio = datetime.strptime("01-01-2000", "%d-%m-%Y")
dataFim = datetime.strptime("01-01-2003", "%d-%m-%Y")

# Chamar a função para calcular a pentada usando a geometria do KML
df_pentada = calcula_pentada(dataInicio, dataFim, geojson_geometry)

# Salvar o DataFrame no arquivo Excel
df_pentada.to_excel('pentada_areaDesmatada1.xlsx', index=False)


ValueError: Length of values (216) does not match length of index (72)

In [21]:
import geopandas as gpd
import ee
import pandas as pd
from datetime import datetime

# Inicializar a API do Google Earth Engine
ee.Authenticate()
ee.Initialize()

# Carregar o arquivo KML usando GeoPandas
kml_file = "/home/talita/Documentos/calculo_climatologia/calculo_python/kml/areaDesmat_Area1.kml"
gdf = gpd.read_file(kml_file, driver='KML')

# Extraindo a geometria da área de interesse (supondo que há uma única geometria no KML)
area_de_interesse = gdf.geometry[0]

# Converter a geometria para o formato GeoJSON que o GEE pode entender
coords = [list(point)[:2] for point in area_de_interesse.exterior.coords]
geojson_geometry = ee.Geometry.Polygon(coords)

def calcula_pentada(dataInicio, dataFim, geometry):
    # Base utilizada para a coleta de dados
    dataset = ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD').filterDate(dataInicio, dataFim)

    # Selecionar a banda de precipitação
    precipitation = dataset.select('precipitation')

    # Criar um dicionário para armazenar os valores de precipitação por ano
    valores_por_ano = {}
    datas_por_pentada = []  # Lista para armazenar as datas correspondentes a cada pentada

    # Percorrer a lista de imagens e obter as datas e valores de precipitação
    for i in range(precipitation.size().getInfo()):
        image = ee.Image(precipitation.toList(precipitation.size()).get(i))
        data = ee.Date(image.get('system:time_start')).format('yyyy-MM-dd').getInfo()  # Extrai a data
        ano = data.split('-')[0]

        # Calcular o valor de precipitação na área de interesse (definida pelo KML)
        valor = image.reduceRegion(ee.Reducer.sum(), geometry).get('precipitation').getInfo()

        # Armazenar o valor no dicionário
        if ano not in valores_por_ano:
            valores_por_ano[ano] = []
        valores_por_ano[ano].append(valor)
        
        # Adicionar a data correspondente à pentada
        datas_por_pentada.append(f"{data} ({ano})")  # Formato: "YYYY-MM-DD (YYYY)"

    # Criar um DataFrame a partir do dicionário de valores por ano
    df_pentada = pd.DataFrame.from_dict(valores_por_ano, orient='index').transpose()

    # Adicionar a coluna de Pentada
    df_pentada["Pentada"] = [f"{i+1}º" for i in range(df_pentada.shape[0])]

    # Renomear as colunas de data
    df_pentada.columns = [f'Data {ano}' if ano.isdigit() else ano for ano in df_pentada.columns]

    # Adicionar a coluna de datas das pentadas
    df_pentada["Datas"] = datas_por_pentada  # Usar a lista de datas formatadas

    return df_pentada

# Definir datas de início e fim 
dataInicio = datetime.strptime("01-01-2010", "%d-%m-%Y")
dataFim = datetime.strptime("01-01-2013", "%d-%m-%Y")

# Chamar a função para calcular a pentada usando a geometria do KML
df_pentada = calcula_pentada(dataInicio, dataFim, geojson_geometry)

# Salvar o DataFrame no arquivo Excel
df_pentada.to_excel('pentada_areaDesmatada1.xlsx', index=False)


ValueError: Length of values (216) does not match length of index (72)

In [25]:
import geopandas as gpd
import ee
import pandas as pd
from datetime import datetime

# Inicializar a API do Google Earth Engine
ee.Authenticate()
ee.Initialize()

# Carregar o arquivo KML usando GeoPandas
kml_file = "/home/talita/Documentos/calculo_climatologia/calculo_python/kml/areaDesmat_Area1.kml"
gdf = gpd.read_file(kml_file, driver='KML')

# Extraindo a geometria da área de interesse (supondo que há uma única geometria no KML)
area_de_interesse = gdf.geometry[0]

# Converter a geometria para o formato GeoJSON que o GEE pode entender
coords = [list(point)[:2] for point in area_de_interesse.exterior.coords]
geojson_geometry = ee.Geometry.Polygon(coords)

def calcula_pentada(dataInicio, dataFim, geometry):
    # Base utilizada para a coleta de dados
    dataset = ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD').filterDate(dataInicio, dataFim)

    # Selecionar a banda de precipitação
    precipitation = dataset.select('precipitation')

    # Criar um dicionário para armazenar os valores de precipitação por ano
    valores_por_ano = {}
    
    # Percorrer a lista de imagens e obter as datas e valores de precipitação
    for i in range(precipitation.size().getInfo()):
        image = ee.Image(precipitation.toList(precipitation.size()).get(i))
        data = ee.Date(image.get('system:time_start')).format('yyyy-MM-dd').getInfo()  # Extrai a data
        ano = data.split('-')[0]

        # Calcular o valor de precipitação na área de interesse (definida pelo KML)
        valor = image.reduceRegion(ee.Reducer.sum(), geometry).get('precipitation').getInfo()

        # Armazenar o valor e a data no dicionário
        if ano not in valores_por_ano:
            valores_por_ano[ano] = []
        valores_por_ano[ano].append(valor)

    # Criar um DataFrame a partir do dicionário de valores por ano
    df_pentada = pd.DataFrame.from_dict(valores_por_ano, orient='index').transpose()

    # Adicionar uma coluna de Pentada com a contagem correta
    df_pentada["Pentada"] = [f"{i+1}º" for i in range(df_pentada.shape[0])]

    # Adicionar a coluna de datas das pentadas, renomeando as colunas
    datas_por_pentada = [f'{ano}' for ano in df_pentada.columns[:-1]]
    df_pentada.columns = datas_por_pentada + ['Pentada']

    return df_pentada

# Definir datas de início e fim 
dataInicio = datetime.strptime("01-01-2000", "%d-%m-%Y")
dataFim = datetime.strptime("01-01-2015", "%d-%m-%Y")

# Chamar a função para calcular a pentada usando a geometria do KML
df_pentada = calcula_pentada(dataInicio, dataFim, geojson_geometry)

# Salvar o DataFrame no arquivo Excel
df_pentada.to_excel('pentada_areaDesmatada1.xlsx', index=False)
